## Inspections and Enforcement

<b> TO DO: </b>
<ul> <li>get values out of the year column to create multiple DFs using those years.</li>
<li>Create Year Column on PROD.gisadmin.Violations Table.</li>
<li>also can create a dictionary with those years in there.</li>
<li>use dask to create a lot of DFs for processing.</li>
<li>Add GeoSpatial Data to All Permits for Historical Permits.</li>
<li>Remove Permits from the DB and upload the new Permits from MGO and AS400 Data.</li>
</ul>

In [ ]:
#Use arcNew environment with this notebook.
#Current Location: C:\ProgramData\Anaconda3\envs\arcNew\python.exe
import arcpy as arc
import os
from shutil import copy2
import pandas as pd
import numpy as np
from arcgis import features
from datetime import datetime as dt
from pathlib import Path
# import geopandas as gpd

In [ ]:
arc.env.overwriteOutput = True
arc.env.outputZFlag = 'Disabled' #To remove z data from parcel fabric due to it being a polygonZ
arc.env.outputMFlag = 'Disabled'
arc.env.qualifiedFieldNames = False
now = dt.now()
mStr = now.strftime('%m%Y')
dStr = now.strftime('%m_%d')
uPath = Path.home()
locFolders = ['Processing', 'Review']
if uPath.exists():
    for x in locFolders:
        a = Path(uPath / 'GIS' / x)
        if a.exists():
            print(f'{a} already exists.')
        else:
            a.mkdir(parents=True)
            print(f'{a} has been created.')
else:
    pass

gisPath = uPath / 'GIS'
lPath = [f for f in gisPath.glob('*')]
netDir = Path(r'\\kcdp-1\KCGIS\MasterGISFiles\Ben')
netDB = netDir / 'GISPro' / 'SDE Connections'
# print(netDB)

### Create Folders for Violations Review Data

In [ ]:
#Create Folders for Permits Data
vFolder = [f for f in lPath if f.name == 'Processing'][0]
vProcessing = vFolder / 'Violations' / f'{dStr}'
if vProcessing.exists() == True:
    print(f'{vProcessing} already exist.')
else:
    vProcessing.mkdir(parents=True)
    print(f'Created {vProcessing}.')

vFR = [f for f in lPath if f.name == 'Review'][0]
vReview = vFR / 'Violations' / f'{dStr}'
if vReview.exists() == True:
    print(f'{vReview} already exist')
else:
    vReview.mkdir(parents=True)
    print(f'Created {vReview}')

### Create New File GeoDatabase and corresponding Feature Datasets.

In [ ]:
iE = netDB / 'PROD-WA.sde'
sr = arc.Describe(f'{iE / "PROD.GISADMIN.SheriffSales"}').spatialReference
outGDB = gisPath / vFolder / f'Data_{mStr}.gdb'
# dsA = gisPath / pFolder / f'{outGDB}.gdb'
locGDB = outGDB / f'Daily_{dStr}'
if arc.Exists(f'{outGDB}'):
    print("GDB already exists.")
else:
    arc.CreateFileGDB_management(f'{vFolder}', f'{outGDB.name}')
    print(f'Created File GeoDatabase at {outGDB.parent}')

time.sleep(2)

if arc.Exists(f'{locGDB}'):
    print(f'{locGDB.name} already exists')
else:
    arc.CreateFeatureDataset_management(f'{locGDB.parent}', f'{locGDB.name}', sr)
    print(f'{locGDB.name} Dataset has been created')

### Create Violations Feature Class

<b>Does not work with Parcel Fabric due to bug with 10.5.1</b>

In [ ]:
dbViol = 'PROD.gisadmin.Violations'
# dbViolD = 'PROD.gisadmin.violations_description'
dbAdd = 'PROD.ADDRESSINGADMIN.Addressing'
dbSufP = 'PROD.GISADMIN.Suffix_Parcels'
dbMPTab = 'PROD.gisadmin.MGOPermits'
dbAPTab = 'PROD.gisadmin.All_Permits'

In [ ]:
#create paths to the data instead of using env.workspace makes it so that I can use arc Walk
arc.env.workspace = f'{iE}'

fcList = [dbAdd, dbSufP]
tList = [dbMPTab, dbAPTab, dbViol]
cParcels = []
cTables = []
for dirpath, dirnames, filenames in arc.da.Walk():
    for f in filenames:
        if f in fcList:
            a  = f.split('.')[-1]
            b = f'{a}_{dStr}'
            if arc.Exists(f'{locGDB / b}'):
                cParcels.append(locGDB.joinpath(b))
                print(f'{b} already exists')
            else:
                arc.FeatureClassToFeatureClass_conversion(f, f'{locGDB}', f'{b}')
                cParcels.append(locGDB.joinpath(b))
                print (f'{b} has been copied')
for dirpath, dirnames, filenames in arc.da.Walk():
    for f in filenames:
        if f in tList:
            a  = f.split('.')[-1]
            b = f'{a}_{dStr}' #Tables need to have a date string since they won't be under the dataset
            # c = locGDB.parent
            if arc.Exists(f'{outGDB / b}'):
                cTables.append(outGDB.joinpath(b))
                print(f'{b} already exists')
            else:
                arc.TableToTable_conversion(f, f'{outGDB}', f'{b}')
                cTables.append(outGDB.joinpath(b))
                print (f'{b} has been copied')
# print(cParcels)
# print(cTables)

### Copy Parcel Fabric to local folder

In [ ]:
#currently using env.workspace to move parcelfabric needs to be changed to use path
dbWS = netDB / 'MAPPINGADMIN.sde' / 'PROD.MAPPINGADMIN.ParcelEditing'
arc.env.workspace = f'{dbWS}'
pfParcels = 'PROD.MAPPINGADMIN.ParcelFabric_Parcels'
exp = "TYPE = 7 AND Historical = 0"
a = pfParcels.split('.')[-1]
b = f'{a}_{dStr}'
# print(f'{iEPath[1]}')
if arc.Exists(f'{locGDB / b}'):
    cParcels.append(locGDB.joinpath(b))
    print(f'{b} already exists')
else:
    print(f'Copying Parcel Fabric to {locGDB}')
    #print(a)
    # g = f'{f}'
    #print(g)
    arc.FeatureClassToFeatureClass_conversion(pfParcels, f'{locGDB}', b, where_clause=exp)
    cParcels.append(locGDB.joinpath(b))
    print(f'Finished Copying {b} to {locGDB}')

Create DataFrame for processing from Parcel Fabric Points

In [ ]:
#Create Points out of polygons in ParcelFabric
pfFC = [f for f in cParcels if f.name.startswith('ParcelFabric') == True][0]
# print(pfFC)
pfPoints = locGDB / f'pfPoints_{dStr}'
# print(pfPoints)
if arc.Exists(f'{pfPoints}'):
    print(f'{pfPoints.name} already exists.')
else:
    arc.FeatureToPoint_management(str(pfFC), f'{pfPoints}', "INSIDE")
    print(f'{pfPoints.name} has been created.')

In [ ]:
pfFields = [f.name for f in arc.ListFields(pfFC)]
# print(pfFields)
pf_df = features.GeoAccessor.from_featureclass(pfPoints, fields=['Name'])
print('Parcel Fabric DataFrame created.')
# pf_df.shape

Create DataFrame from Parcel Fabric Polygons

In [ ]:
# pfFC = [f for f in cParcels if f.name.startswith('ParcelFabric') == True][0]
ppf_df = features.GeoAccessor.from_featureclass(pfFC, fields=['Name'])
print('Parcel Fabric DataFrame created.')

#### Create Violations Table DataFrame

In [ ]:
#Copy table to the locGDB from ESRIDB
vTab = [f for f in cTables if f.name.startswith('Violations') == True][0]

v_df = features.GeoAccessor.from_table(f'{vTab}', skip_nulls=False)
v_df.drop(columns='OBJECTID', inplace=True)

In [ ]:
def newyear(field):
    if int(field) > 80 and int(field) < 100:
        return "19" + str(field)
    elif int(field) >= 0 and int(field) <=9:
        return "200" + str(field)
    elif int(field) > 10 and int(field) < 80:
        return "20" + str(field)

Apply newyear function and sort by Year.

In [ ]:
v_df.YEAR = v_df.YEAR.apply(newyear)

In [ ]:
v_df.shape

In [ ]:
v_df.columns

Merge with Parcel Fabric Points to obtain GeoSpatial Data

In [ ]:
vLoc = locGDB / f'Violation_{dStr}'
df_gf = pf_df.merge(v_df, how='outer', left_on='Name', right_on='PARCELID', indicator=True)

In [ ]:
# df.to_csv(f'{gisPath / "Corrupted.csv"}')
df_gf = df_gf.loc[df_gf._merge != 'left_only'].copy(deep=True)
df_gf.drop(columns='Name', inplace=True)

In [ ]:
# df_gf.Name = df_gf.Name.astype(str)
# df_gf.VIOLATIONSTATUS = df_gf.VIOLATIONSTATUS.astype(str)
# df_gf.VIOLATIONTYPE = df_gf.VIOLATIONTYPE.astype(str)
# df_gf.VIOLATIONDESC = df_gf.VIOLATIONDESC.astype(str)
# df_gf.YEAR = df_gf.YEAR.astype(str)

In [ ]:
df_gf.reset_index(drop=True, inplace=True)

In [ ]:
# fV = df_gf.spatial.to_featureset()
# fV.fields

In [ ]:
# fV.spatial_reference
# a = fV.features
# f = []
# for feat in fV.features:
#     a = feat.geometry
#     f.append(a)
# l = gisPath / 'op.txt'
# l.touch()
# g = l.open('a', newline='\n')
# for x in f:
#     g.write(str(x))

In [ ]:
# fV.save(f'{outGDB}', 'ViolTT')

In [ ]:
# df_gf.drop(columns='SHAPE', inplace=True)
# df_gf.spatial.to_table(f'{outGDB / "VTest"}')

Merge Violation Suffix Parcels with Parcels with Suffix

In [ ]:
df_gf.columns

Merge Parcel With Suffix with Violations with Suffix.

In [ ]:
#Import ParcelsWithSuffix and create DataFrame
psLoc = locGDB / f'ParcelWithSuffix_{dStr}'
# psLoc = locGDB.parent / 'Daily_05_04' / 'ParcelWithSuffix'
psDF = features.GeoAccessor.from_featureclass(psLoc, fields=['Name'])

ps_df = df_gf[(df_gf._merge == 'right_only') & (~df_gf.PARCELID.str.contains('-00001', na=False))].copy(deep=True)
ps_df.drop(columns=['_merge','SHAPE'], inplace=True)

m_ps = psDF.merge(ps_df, how='outer', left_on='Name', right_on='PARCELID', indicator=True)

Export the Parcel #s that don't match with the Parcel With Suffix Layer

In [ ]:
mDM = vReview / 'PS_NF.csv' #NF = Not Found
m_ps[m_ps._merge == 'right_only'].to_csv(mDM)

In [ ]:
#Create new DF with matching Parcel #s 
eMP = m_ps[m_ps._merge != 'left_only'].copy(deep=True)
eMP.drop(columns=['Name','_merge'], inplace=True)

Export to csv the Parcel #s that don't have a match in Parcel Fabric and remove them

In [ ]:
pNF = vReview / 'Parcel_NF.csv' #NF = NOT FOUND
nf_df = df_gf[(df_gf._merge == 'right_only') & (df_gf.PARCELID.str.contains('-00001'))].copy(deep=True)
nf_df.to_csv(pNF, index=False)
nf_df.drop(columns='_merge', inplace=True)

#Create new DF without suffix parcels and the NF parcels, will merge this back in later
ns_df = df_gf[(df_gf._merge != 'left_only') & (df_gf.PARCELID.str.contains('-00001'))].copy(deep=True)
ns_df.drop(columns='_merge', inplace=True)

In [ ]:
ns_df.shape

In [ ]:
eMP.shape

In [ ]:
print(ns_df.columns)
print(eMP.columns)

Combine the violations with suffix and the violations table to create Historical Feature Class

In [ ]:
hv_df = pd.concat([ns_df,eMP], axis=0)

In [ ]:
hv_df.rename(columns={'PARCELID':'NAME'}, inplace=True)

Export the DataFrame to locGDB as a feature class. Contains the Historical Violations Data

In [ ]:
#check to see why there are null years
nYcsv = vReview / 'Viol_Null.csv'
hv_df[hv_df.YEAR == ' '].to_csv(nYcsv, index=False)

In [ ]:
hv_df.SHAPE.fillna(np.nan, inplace=True)
hv_df.YEAR.fillna(np.nan, inplace=True)
hv_df.reset_index(drop=True, inplace=True)

In [ ]:
fsr = {'x': None, 'y': None, 'spatialReference':{'wkid':26957}}
# fr = {'spatialReference':{'wkid':26957}}
hv_df.SHAPE = hv_df.SHAPE.apply(lambda x: fsr if x == 0 else x)

In [ ]:
vCurrent = locGDB / f'HViolation_{dStr}'
# geo_df = gpd.GeoDataFrame(hv_df, geometry='SHAPE', crs=26957)
# vSTest = locGDB / f'HViol_{dStr}'
# vSTest = gisPath / 'lol.shp'
# geo_df.to_file(f'{vSTest}')
# gg = features.GeoAccessor.from_geodataframe(geo_df)
# hv_df.spatial.to_featureclass(f'{vSTest}', sanitize_columns=False)
hv_df.spatial.to_featureclass(f'{vCurrent}', sanitize_columns=False)

#### Create Current Violations by removing duplicates

Export Null Dates to CSV for review

In [ ]:

dup_v = hv_df.copy(deep=True)
dup_v = dup_v.sort_values(by='YEAR')


In [ ]:
dup_v = dup_v.loc[dup_v.YEAR.notnull()]

 Remove duplicates based on the most recent year.

In [ ]:
dup_v.drop_duplicates(subset=['PARCELID'], keep='last', ignore_index=True, inplace=True)

Export Feature class to locGDB to be used with REST Server